In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import os

In [7]:
df = pd.read_csv('tweets_stocks_combined_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2669 entries, 0 to 2668
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     2669 non-null   float64
 1   cleaned_text           2669 non-null   object 
 2   favorites              2669 non-null   int64  
 3   retweets               2669 non-null   int64  
 4   date                   2669 non-null   object 
 5   tweet_datetime         2669 non-null   object 
 6   date_part              2669 non-null   object 
 7   time_part              2669 non-null   object 
 8   hour                   2669 non-null   int64  
 9   year                   2669 non-null   int64  
 10  month                  2669 non-null   int64  
 11  cleaned_text_2         2669 non-null   object 
 12  datetime_5mins_after   2669 non-null   object 
 13  price_5mins_after      2669 non-null   float64
 14  datetime_now           2669 non-null   object 
 15  pric

In [4]:
# model_a_df = df.dropna(subset=['cleaned_text'])

In [8]:
model_a_df

,id,cleaned_text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,cleaned_text_2,datetime_5mins_after,price_5mins_after,datetime_now,price_now,5mins_price_diff_abs,5mins_price_diff_perc,prev_30mins_prices
0,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,thank you rand,2017-11-28 10:55:00,261.085000,2017-11-28 10:50:00,261.100000,-0.015000,-0.000057,"[260.96, 261.03, 261.01, 261.015, 261.04, 261...."
1,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,join me live from fort myer in arlington virgi...,2017-08-22 09:05:00,243.670000,2017-08-22 09:00:00,243.670000,0.000000,0.000000,"[243.76, 243.79, 243.85, 243.86, 243.81, 243.7..."
2,8.939700e+17,Thank you Nicole!,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,thank you nicole,2017-05-09 07:06:00,239.920000,2017-05-09 07:01:00,239.875000,0.045000,0.000188,"[239.73, 239.73, 239.73, 239.73, 239.73, 239.7..."
3,8.819770e+17,Thank you to Shawn Steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,thank you to shawn steel for the nice words on,2017-03-08 04:49:00,236.913333,2017-03-08 04:44:00,236.880000,0.033333,0.000141,"[236.84, 236.84, 236.84, 236.85333333333332, 2..."
4,8.778460e+17,Great night in Iowa - special people. Thank you!,56446,8039,2017-06-22 11:11:00,2017-06-22 19:11:00,2017-06-22,19:11:00,19,2017,6,great night in iowa special people thank you,2017-06-22 19:16:00,242.905000,2017-06-22 19:11:00,242.880000,0.025000,0.000103,"[242.85, 242.85, 242.85, 242.85, 242.85, 242.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2664,9.990960e+17,If the person placed very early into my campai...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,if the person placed very early into my campai...,2018-05-23 09:18:00,271.100000,2018-05-23 09:13:00,271.040000,0.060000,0.000221,"[271.18, 271.16, 271.18, 271.15, 271.08, 271.0..."
2665,9.874600e+17,So General Michael Flynn’s life can be totally...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,so general michael flynn’s life can be totally...,2018-04-20 18:39:00,266.866667,2018-04-20 18:34:00,266.820000,0.046667,0.000175,"[266.74625000000003, 266.7475, 266.74875, 266...."
2666,9.870960e+17,"My thoughts, prayers and condolences are with ...",62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,my thoughts prayers and condolences are with t...,2018-04-20 06:35:00,268.715000,2018-04-20 06:30:00,268.620000,0.095000,0.000354,"[268.77, 268.74333333333334, 268.7166666666666..."
2667,9.863570e+17,Today’s Court decision means that Congress mus...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,today’s court decision means that congress mus...,2018-04-18 05:39:00,270.600000,2018-04-18 05:34:00,270.600000,0.000000,0.000000,"[270.73, 270.7, 270.72749999999996, 270.755, 2..."


# Model A (only word vectors)

In [9]:
model_a_X = model_a_df.loc[:, 'cleaned_text_2']
model_a_y = model_a_df.loc[:, '5mins_price_diff_perc']

In [10]:
model_a_X_train, model_a_X_test, model_a_y_train, model_a_y_test = train_test_split(model_a_X, model_a_y, test_size=0.33, random_state=42)

In [11]:
model_a_corpus_list = []

for i in model_a_X_train:
    model_a_corpus_list.append(i.split())

In [12]:
model_a_word2vec_model = Word2Vec(model_a_corpus_list, min_count=1, size=100)
model_a_pretrained_weights = model_a_word2vec_model.wv.vectors

In [13]:
model_a_num_words = [len(i) for i in model_a_corpus_list]
model_a_longest_sentence_len = max(model_a_num_words)

In [14]:
def sentence_to_indices_padded(sentences, longest_sentence_len):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in model_a_word2vec_model.wv.vocab:
                indices.append(model_a_word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [15]:
model_a_X_train_padded = sentence_to_indices_padded(model_a_X_train, model_a_longest_sentence_len)
model_a_X_test_padded = sentence_to_indices_padded(model_a_X_test, model_a_longest_sentence_len)

In [12]:
embeddings_index = {}
f = open('glove/glove.twitter.27B.100d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [13]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(model_a_X_train)
sequences = tokenizer.texts_to_sequences(model_a_X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5893 unique tokens.


C:\anaconda\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [14]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

{'the': 2435, 'to': 1390, 'and': 1380, 'of': 1116, 'a': 867, 'in': 788, 'is': 741, 'for': 604, 'great': 499, 'on': 498, 'our': 470, 'will': 431, 'are': 413, 'with': 397, 'be': 393, 'we': 392, 'i': 389, 'that': 384, 'it': 331, 'amp': 318, 'you': 300, 'they': 278, 'at': 266, 'was': 264, 'have': 253, 'people': 224, 'this': 221, 'he': 209, 'all': 204, 'my': 196, 'has': 193, 'by': 193, 'very': 191, 'not': 185, 'us': 175, 'who': 171, 'so': 169, 'from': 157, 'just': 156, 'thank': 155, 'country': 152, 'big': 146, 'as': 140, 'news': 136, 'now': 133, 'many': 129, 'president': 129, 'america': 129, 'fake': 127, 'no': 126, 'today': 123, 'an': 121, 'out': 121, 'more': 118, 'his': 113, 'their': 112, 'get': 110, 'democrats': 109, 'do': 108, 'but': 107, 'there': 106, 'about': 104, 'trump': 104, 'time': 102, 'american': 101, 'or': 101, 'been': 99, 'much': 98, 'new': 97, 'jobs': 96, 'border': 96, 'your': 95, 'trade': 95, 'must': 94, 'what': 90, 'tax': 89, 'make': 86, 'up': 85, 'me': 85, 'vote': 85, 'back

In [15]:
w2c = dict()
for item in model_a_word2vec_model.wv.vocab:
    w2c[item]=model_a_word2vec_model.wv.vocab[item].count
w2c_sorted=dict(sorted(w2c.items(), key=lambda x: x[1],reverse=True))
print(w2c_sorted)

{'the': 2435, 'to': 1390, 'and': 1380, 'of': 1116, 'a': 867, 'in': 788, 'is': 741, 'for': 604, 'great': 499, 'on': 498, 'our': 470, 'will': 431, 'are': 413, 'with': 397, 'be': 393, 'we': 392, 'i': 389, 'that': 384, 'it': 331, 'amp': 318, 'you': 300, 'they': 278, 'at': 266, 'was': 264, 'have': 253, 'people': 224, 'this': 221, 'he': 209, 'all': 204, 'my': 196, 'has': 193, 'by': 193, 'very': 191, 'not': 185, 'us': 175, 'who': 171, 'so': 169, 'from': 157, 'just': 156, 'thank': 155, 'country': 152, 'big': 146, 'as': 140, 'news': 136, 'now': 133, 'many': 129, 'president': 129, 'america': 129, 'fake': 127, 'no': 126, 'today': 123, 'an': 121, 'out': 121, 'more': 118, 'his': 113, 'their': 112, 'get': 110, 'democrats': 109, 'do': 108, 'but': 107, 'there': 106, 'about': 104, 'trump': 104, 'time': 102, 'american': 101, 'or': 101, 'been': 99, 'much': 98, 'new': 97, 'jobs': 96, 'border': 96, 'your': 95, 'trade': 95, 'must': 94, 'what': 90, 'tax': 89, 'make': 86, 'up': 85, 'me': 85, 'vote': 85, 'back

In [16]:
glove_word_count = dict(tokenizer.word_counts)
glove_sorted=dict(sorted(glove_word_count.items(), key=lambda x: x[1],reverse=True))
print(glove_sorted)

{'the': 2435, 'to': 1390, 'and': 1380, 'of': 1116, 'a': 867, 'in': 788, 'is': 741, 'for': 604, 'great': 499, 'on': 498, 'our': 470, 'will': 431, 'are': 413, 'with': 397, 'be': 393, 'we': 392, 'i': 389, 'that': 384, 'it': 331, 'amp': 318, 'you': 300, 'they': 278, 'at': 266, 'was': 264, 'have': 253, 'people': 224, 'this': 221, 'he': 209, 'all': 204, 'my': 196, 'has': 193, 'by': 193, 'very': 191, 'not': 185, 'us': 175, 'who': 171, 'so': 169, 'from': 157, 'just': 156, 'thank': 155, 'country': 152, 'big': 146, 'as': 140, 'news': 136, 'now': 133, 'many': 129, 'president': 129, 'america': 129, 'fake': 127, 'no': 126, 'today': 123, 'an': 121, 'out': 121, 'more': 118, 'his': 113, 'their': 112, 'get': 110, 'democrats': 109, 'do': 108, 'but': 107, 'there': 106, 'about': 104, 'trump': 104, 'time': 102, 'american': 101, 'or': 101, 'been': 99, 'much': 98, 'new': 97, 'jobs': 96, 'border': 96, 'your': 95, 'trade': 95, 'must': 94, 'what': 90, 'tax': 89, 'make': 86, 'up': 85, 'me': 85, 'vote': 85, 'back

In [17]:
words_w2v = list(w2c.keys())
words_gloves = list(glove_word_count.keys())
extra = []
for word in words_w2v:
    if word not in words_gloves:
        extra.append(word)
print(extra)

[]


In [27]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)
vocab_size_glove, embedding_size_glove = embedding_matrix.shape
print(vocab_size_glove)
print(embedding_size_glove)
print(count)
print(skipped_words)
embedding_layer_glove = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=model_a_longest_sentence_len,
                            trainable=False)

5894
100
916
['don’t', 'foxandfriends', 'didn’t', 'it’s', 'can’t', '“the', 'strzok', 'doesn’t', 'isn’t', 'loudobbs', 'seanhannity', 'councel', 'that’s', 'aren’t', 'couldn’t', 'trump”', 'kavanaugh', 'wasn’t', 'he’s', 'won’t', 'magarally', 'foxbusiness', '“trump', 'america’s', 'we’ve', 'dhsgov', 'petestauber', 'tuckercarlson', 'i’ve', 'country’s', 'emmanuelmacron', 'mariabartiromo', '“i', 'today’s', 'senatemajldr', 'usmca', 'antitrump', 'states”', 'deptvetaffairs', 'haspel', 'oreillyfactor', 'abeshinzo', 'maralago', 'taxreform', 'people’s', '“we', 'we’re', 'speakerryan', 'wouldn’t', 'i’m', 'fitton', 'ohr’s', 'you’re', 'they’re', 'they’ve', 'wyou', 'ivankatrump', 'i’ll', '“it', '“mr', 'ossoff', 'icegov', 'americafirst', 'rosendale', 'world’s', 'jobsnotmobs', 'breitbartnews', '“why', 'gopleader', 'hillary’s', '“you', 'tillerson', 'mcgahn', '“president', '“justice”', 'denuclearization', '“this', 'ingrahamangle', 'jimhagedornmn', 'brennan’s', 'sheriff’s', 'nonmonetary', 'votekarenhandel', 'h

In [28]:
def create_model_a(pretrained_weights, longest_sentence_len):
    global embedding_layer_glove
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
    model.add(embedding_layer_glove)
    #model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [29]:
model_a = create_model_a(model_a_pretrained_weights, model_a_longest_sentence_len)

In [30]:
model_a.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [31]:
model_a.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 157, 100)          589400    
_________________________________________________________________
LSTM1 (LSTM)                 (None, 157, 4)            1680      
_________________________________________________________________
Dropout1 (Dropout)           (None, 157, 4)            0         
_________________________________________________________________
LSTM2 (LSTM)                 (None, 4)                 144       
_________________________________________________________________
Dropout2 (Dropout)           (None, 4)                 0         
_________________________________________________________________
Dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dropout (Dropout)            (None, 4)                

In [32]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_a_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a.fit(model_a_X_train_padded, model_a_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

Train on 1425 samples, validate on 357 samples
Epoch 1/50
1376/1425 [===========================>..] - ETA: 0s - loss: 0.7520 - mae: 0.8478
Epoch 00001: val_loss improved from inf to 0.43913, saving model to ./model_a_checkpoint/10112020 1819h.h5
1425/1425 [==============================] - 5s 3ms/sample - loss: 0.7425 - mae: 0.8419 - val_loss: 0.4391 - val_mae: 0.6627
Epoch 2/50
1376/1425 [===========================>..] - ETA: 0s - loss: 0.3342 - mae: 0.5624
Epoch 00002: val_loss improved from 0.43913 to 0.21717, saving model to ./model_a_checkpoint/10112020 1819h.h5
1425/1425 [==============================] - 1s 743us/sample - loss: 0.3317 - mae: 0.5601 - val_loss: 0.2172 - val_mae: 0.4660
Epoch 3/50
1376/1425 [===========================>..] - ETA: 0s - loss: 0.1900 - mae: 0.4197
Epoch 00003: val_loss improved from 0.21717 to 0.11213, saving model to ./model_a_checkpoint/10112020 1819h.h5
1425/1425 [==============================] - 1s 740us/sample - loss: 0.1881 - mae: 0.4174 - v

In [33]:
y_pred = model_a.predict(model_a_X_test_padded)

In [34]:
print(y_pred)

[[-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.0172435 ]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724361]
 [-0.01724

In [35]:
from sklearn import metrics

model_a.evaluate(model_a_X_test_padded, model_a_y_test)

878/878 [==============================] - 0s 376us/sample - loss: 2.9841e-04 - mae: 0.0173


[0.000298409072583083, 0.017268343]

# Model B (word vectors + price history)

In [54]:
model_b_df = df.dropna(subset=['cleaned_text', 'prev_30mins_prices'])

model_b_X = model_b_df.loc[:, ['cleaned_text', 'prev_30mins_prices']]
model_b_y = model_b_df.loc[:, '5mins_price_diff_perc']

model_b_X_train, model_b_X_test, model_b_y_train, model_b_y_test = train_test_split(model_b_X, model_b_y, test_size=0.33, random_state=42)

model_b_X_train_text = model_b_X_train.iloc[:, 0]
model_b_X_train_price_history = model_b_X_train.iloc[:, 1]
model_b_X_test_text = model_b_X_test.iloc[:, 0]
model_b_X_test_price_history = model_b_X_test.iloc[:, 1]
                           
model_b_corpus_list = []

for i in model_b_X_train_text:
    model_b_corpus_list.append(i.split())
    
model_b_word2vec_model = Word2Vec(model_b_corpus_list, min_count=1, size=100)
model_b_pretrained_weights = model_b_word2vec_model.wv.vectors

model_b_num_words = [len(i) for i in model_b_corpus_list]
model_b_longest_sentence_len = max(model_b_num_words)

model_b_X_train_padded = sentence_to_indices_padded(model_b_X_train_text, model_b_longest_sentence_len)
model_b_X_test_padded = sentence_to_indices_padded(model_b_X_test_text, model_b_longest_sentence_len)

model_b_X_train_input = [model_b_X_train_padded, np.array(model_b_X_train_price_history)]
model_b_X_test_input = [model_b_X_test_padded, model_b_X_test_price_history]

In [55]:
type(np.array(model_b_X_train_input[1]))

numpy.ndarray

In [56]:
def create_model_b(pretrained_weights, longest_sentence_len, price_history_shape):
    vocab_size, embedding_size = pretrained_weights.shape
    
    # word vectors model
    model1_input = layers.Input(shape=longest_sentence_len, dtype='int32', name='sentence_index_input')
    model1 = layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False)(model1_input)  
    model1 = layers.LSTM(4, return_sequences=True, name='model1_LSTM1')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout1')(model1)
    model1 = layers.LSTM(4, return_sequences=False, name='model1_LSTM2')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout2')(model1)
    
    # price history model
    model2_input = layers.Input(shape=price_history_shape, dtype='float32', name='price_history_input')
    model2 = layers.LSTM(4, return_sequences=True, name='model2_LSTM1')(model2_input)
    model2 = layers.Dropout(0.25,name='model2_dropout1')(model2)
    model2 = layers.LSTM(4, return_sequences=False, name='model2_LSTM2')(model2)
    model2 = layers.Dropout(0.25,name='model2_dropout2')(model2)
    
    model_concat = layers.concatenate([model1, model2])
    model_concat = layers.Dense(4,name='Dense',activation='relu')(model_concat)
    model_concat = layers.Dropout(0.1)(model_concat)
    model_concat = layers.Dense(1,activation='linear')(model_concat)
    
    model = keras.models.Model(inputs=[model1_input, model2_input], outputs = model_concat)
    
    return model

In [57]:
model_b = create_model_b(model_b_pretrained_weights, model_b_longest_sentence_len, (30,1,))

In [58]:
model_b.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [59]:
model_b.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_index_input (InputLaye [(None, 157)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 157, 100)     568600      sentence_index_input[0][0]       
__________________________________________________________________________________________________
price_history_input (InputLayer [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
model1_LSTM1 (LSTM)             (None, 157, 4)       1680        embedding_2[0][0]                
____________________________________________________________________________________________

In [60]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_b_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_b.fit(model_b_X_train_input, model_b_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

ValueError: Error when checking input: expected price_history_input to have 3 dimensions, but got array with shape (1691, 1)